In [ ]:
!pip install tensorflow


In [ ]:
# Install library
!pip install pandas numpy tensorflow scikit-learn

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Daftar nama file dataset
dataset_files = [
    "dataset-ayam.csv",
    "dataset-kambing.csv",
    "dataset-sapi.csv",
    "dataset-telur.csv",
    "dataset-tahu.csv",
    "dataset-ikan.csv",
    "dataset-tempe.csv"
]

# Buat dictionary untuk menyimpan data dari setiap dataset
datasets = {}

# Memuat dan menyimpan setiap dataset ke dalam dictionary
for file in dataset_files:
    category_name = file.split("-")[1].split(".")[0]
    datasets[category_name] = pd.read_csv(file)

# Menggabungkan semua dataset menjadi satu DataFrame
df_all = pd.concat(datasets.values(), ignore_index=True)


In [ ]:
# Pemrosesan data
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_all['Ingredients'].fillna(''))


In [ ]:
# Menampilkan nama kolom yang ada di DataFrame
print(df_all.columns)


Index(['Title', 'Ingredients', 'Steps', 'Loves', 'URL'], dtype='object')


In [ ]:
# Menampilkan jumlah dataset
initial_dataset_count = len(df_all)
print(f"Jumlah dataset sebelum penyaringan: {initial_dataset_count}")


Jumlah dataset sebelum penyaringan: 13647


In [ ]:
# Menampilkan beberapa baris pertama dari DataFrame
print(df_all.head())


                      Title  \
0          Ayam Woku Manado   
1  Ayam goreng tulang lunak   
2          Ayam cabai kawin   
3               Ayam Geprek   
4               Minyak Ayam   

                                         Ingredients  \
0  1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...   
1  1 kg ayam (dipotong sesuai selera jangan kecil...   
2  1/4 kg ayam--3 buah cabai hijau besar--7 buah ...   
3  250 gr daging ayam (saya pakai fillet)--Secuku...   
4  400 gr kulit ayam & lemaknya--8 siung bawang p...   

                                               Steps  Loves  \
0  Cuci bersih ayam dan tiriskan. Lalu peras jeru...      1   
1  Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...      1   
2  Panaskan minyak di dalam wajan. Setelah minyak...      2   
3  Goreng ayam seperti ayam krispi--Ulek semua ba...     10   
4  Cuci bersih kulit ayam. Sisihkan--Ambil 50 ml ...      4   

                                          URL  
0          /id/resep/4473027-ayam-woku-manado  
1

In [ ]:
# Filter resep berdasarkan kata kunci
keywords_to_exclude = ['pedas', 'balado', 'Pedas', 'Geprek', 'Jeruk', 'Sambal', 'Cabai', 'cabai', 'gula', 'kafein', 'cabe', 'minyak', 'asam', 'jeruk', 'lemon']

# Mengatasi nilai-nilai NaN dengan menggantinya dengan string kosong
filtered_df['Ingredients'] = filtered_df['Ingredients'].fillna('')
filtered_df['Title'] = filtered_df['Title'].fillna('')
filtered_df['Steps'] = filtered_df['Steps'].fillna('')

# Gunakan operator & (and) untuk menyaring data
for keyword in keywords_to_exclude:
    filtered_df = filtered_df[~(
        (filtered_df['Ingredients'].str.contains(keyword, case=False)) |
        (filtered_df['Title'].str.contains(keyword, case=False)) |
        (filtered_df['Steps'].str.contains(keyword, case=False))
    )]


In [ ]:
# Menampilkan jumlah dataset yang sudah terfilter
filtered_dataset_count = len(filtered_df)
print(f"Jumlah dataset setelah penyaringan: {filtered_dataset_count}")

Jumlah dataset setelah penyaringan: 934


In [ ]:
# Menampilkan beberapa baris pertama dari dataset yang telah difilter
print(filtered_df.head())

                         Title  \
0        Nugget Ayam Home Made   
1           Nuttget ayam+sayur   
2              Tim Ayam Bawang   
3  Sup kembang tahu+bakso ayam   
4           Nugget ayam simpel   

                                         Ingredients  \
0  1/4 kg ayam d fillet ambil dagingnya aja--1 bu...   
1  1 butir telur--1/2 kg Ayam--1 wortel--4 sdm te...   
2  1 ekor ayam potong sesuai selera (saya potong ...   
3  100 gram kembang tahu kering--2 buah wortel po...   
4  500 gr ayam giling--6 buah bawang merah--4 baw...   

                                               Steps  Loves  \
0  Cuci bersih ayam. Setelah itu potong kecil2. M...      6   
1  Filet ayam ambil bagian dagingnya lalu di cinc...      4   
2  Masukan ayam dan semua bahan kedalam 2ltr air....      6   
3  Rendam kembang tahu dlm air kurleb 1-2 jam.ren...     13   
4  Ulek bawang sampai halus. Sayuran di parut (se...      6   

                                            URL  
0       /id/resep/4472482-nug

In [ ]:
# Reset indeks pada filtered_df
filtered_df = filtered_df.reset_index(drop=True)

In [ ]:
# Membuat matriks TF-IDF untuk dataset yang sudah difilter
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
filtered_tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['Ingredients'])

In [ ]:
# Model machine learning
model = Sequential([
    Dense(128, input_shape=(tfidf_matrix.shape[1],), activation='relu'),
    Dense(64, activation='relu'),
    Dense(tfidf_matrix.shape[0], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Model machine learning
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# Fungsi rekomendasi
def recommend_recipe(title, filtered_df, cosine_similarities):
    idx = filtered_df.index[filtered_df['Title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Ambil 10 resep teratas
    recipe_indices = [i[0] for i in sim_scores]
    return filtered_df.loc[recipe_indices, ['Title', 'Love']]


In [ ]:
# Form input pengguna
user_input = input("Masukkan bahan yang Anda miliki, pisahkan dengan koma: ")


Masukkan bahan yang Anda miliki, pisahkan dengan koma: tahu


In [ ]:
# Membuat vektor TF-IDF dari input pengguna
user_tfidf_matrix = tfidf_vectorizer.transform([user_input])


In [ ]:
# Hitung kesamaan antara input pengguna dan resep dalam dataset yang sudah difilter
user_cosine_similarities = linear_kernel(user_tfidf_matrix, filtered_tfidf_matrix)


In [ ]:
# Dapatkan indeks resep yang direkomendasikan
user_recipe_indices = user_cosine_similarities.argsort()[0][::-1][:10]


In [ ]:
# Reset indeks pada df_all
df_all = df_all.reset_index(drop=True)


In [ ]:
# Tampilkan rekomendasi untuk pengguna
user_recommendations_with_love = df_all[['Title', 'Loves']].iloc[user_recipe_indices]
print("\nResep yang direkomendasikan untuk Anda:")
print(user_recommendations_with_love)

# Pilih resep berdasarkan input pengguna
selected_recipe_index = int(input("\nPilih nomor resep yang ingin Anda lihat langkah-langkahnya: ")) - 1

# Pastikan nomor resep yang dipilih valid
if 0 <= selected_recipe_index < len(user_recommendations_with_love):
    selected_recipe_title = user_recommendations_with_love.iloc[selected_recipe_index]['Title']

    # Dapatkan informasi resep yang dipilih
    selected_recipe_info = df_all[df_all['Title'] == selected_recipe_title]

    # Tampilkan informasi resep yang dipilih
    print(f"\nInformasi untuk resep '{selected_recipe_title}':")
    print("Ingredients:")
    print(selected_recipe_info['Ingredients'].values[0])

    print("\nLangkah-langkah:")
    selected_recipe_steps = selected_recipe_info['Steps'].values[0]
    print(selected_recipe_steps)

    print("\nLoves:")
    print(selected_recipe_info['Loves'].values[0])

else:
    print("\nNomor resep yang dipilih tidak valid.")




Resep yang direkomendasikan untuk Anda:
                                        Title  Loves
652  Ayam goreng korea (korean fried chicken)      4
883                               Buncis ayam      6
720                         Lumpia ayam telur      0
757                         Ayam tahu kemangi      3
640                               Bakwan Ayam      6
674                  Ayam goreng tanpa ungkep      1
622              Ayam dan tempe sambel mentah      5
428                      Opor Ayam Telor Tahu     13
624                            Tahu Baso Ayam     11
704                   Pepes ayam selimut tahu     37

Pilih nomor resep yang ingin Anda lihat langkah-langkahnya: 5

Informasi untuk resep 'Bakwan Ayam':
Ingredients:
1 bungkus Tepung Bumbu Instan Non MSG--Secukupnya Ayam Goreng Suwir--Secukupnya Kol Iris Halus--Secukupnya Daun Bawang Iris--Secukupnya Wortel Parut--Secukupnya Air--Secukupnya Minyak untuk menggoreng--

Langkah-langkah:
Campurkan semua bahan ke dalam wadah. Adu

In [ ]:
!pip install h5py

In [ ]:
import h5py

# Simpan model ke dalam file HDF5
model.save('recommendation_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Download file model
from google.colab import files

files.download('recommendation_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>